In [56]:
from bs4 import BeautifulSoup
import pandas as pd
import html5lib
import re
import json
import requests
from scholarly import scholarly

In [78]:
author_Citation=[]
author_HINDEX_ALL=[]
author_I10_INDEX_ALL=[]
author_year_list=[]
author_year_value_list=[]
author_googleScholar_page_url=[]

In [55]:
def get_url_of_author_page(name):
    try:
        author_data = next(scholarly.search_author(name))
        scholar_Id=(author_data['scholar_id'])
        url='https://scholar.google.com/citations?hl=en&user='+scholar_Id
        author_googleScholar_page_url.append(url)
    except Exception as e:
        author_googleScholar_page_url.append('Not Found')

In [83]:
def scrap_author_googleScholar_data(url):
    cnt = requests.get(url)
    htmlContent = cnt.content
    soup = BeautifulSoup(htmlContent, 'html.parser')
    graph_values=[]
    graph_years=[]
    valueName=['CITATIONS_ALL','HINDEX_ALL','I10_INDEX_ALL']
    li=[]
    rqrd_td=[]
    rdrd_table=soup.find("table", attrs={"id": "gsc_rsb_st"})
    rqrd_tr = rdrd_table.find_all("tr")
    for x in range(len(rqrd_tr)):
        if x==0:
            continue
        rqrd_td.append(rqrd_tr[x].find_all('td'))
    for x in range(len(rqrd_td[1])):
            li.append(str(rqrd_td[x][1].text).strip())
    for x in range(len(li)):
        if x==0:
            author_Citation.append(li[x])
        elif x==1:
            author_HINDEX_ALL.append(li[x])
        elif x==2:
            author_I10_INDEX_ALL.append(li[x])     
    div_tag=soup.find_all("div",attrs={"id":"gsc_prf_in"})
    a_tag=soup.find_all("a",attrs={"class":"gsc_g_a"})
    for x in a_tag:
        graph_values.append(str(x.text).strip())
    author_year_value_list.append(graph_values)
    span_tag=soup.find_all("span",attrs={"class":"gsc_g_t"})
    for x in span_tag:
        graph_years.append(str(x.text).strip())
    author_year_list.append(graph_years)

In [85]:
def scrap_goggle_scholor_page(name):
    author_Citation.clear()
    author_HINDEX_ALL.clear()
    author_I10_INDEX_ALL.clear()
    author_year_list.clear()
    author_year_value_list.clear()
    author_googleScholar_page_url.clear()
    df=pd.read_csv(name)
    Professor_Name_list = df["Professor Name"].tolist()
    print("Extracting Urls of Author's Google scholor page.....")
    for i in range(len(Professor_Name_list)):
        if i==10:
            break
        get_url_of_author_page(Professor_Name_list[i])
    print(author_googleScholar_page_url)
    print("Extracting Data from Google Scholor Page......")
    for i in range(len(author_googleScholar_page_url)):
        if author_googleScholar_page_url[i]=='Not Found':
            author_Citation.append(None)
            author_HINDEX_ALL.append(None)
            author_I10_INDEX_ALL.append(None)
            author_year_list.append(None)
            author_year_value_list.append(None)
            continue
        scrap_author_googleScholar_data(author_googleScholar_page_url[i])
    print(len(author_googleScholar_page_url))
    print(len(author_Citation))
    print(len(author_HINDEX_ALL))
    print(len(author_I10_INDEX_ALL))
    print("Creating Csv File.......")
    Scholar_df = pd.DataFrame({'Author_URL':author_googleScholar_page_url,'CITATIONS_ALL':author_Citation,'HINDEX_ALL':author_HINDEX_ALL,'I10_INDEX_ALL': author_I10_INDEX_ALL})
    Starting_year=2021
    for i in range(2021-1979):
        indx=str(Starting_year)
        Scholar_df[indx]=0
        Starting_year-=1
    for column in Scholar_df:
        for i in range(len(author_year_list)):
            if author_year_list[i]==None:
                continue
            for j in range(len(author_year_list[i])):
                if column==author_year_list[i][j]:
                    columnSeriesObj = Scholar_df[column]
                    columnSeriesObj.values[i]=author_year_value_list[i][j]
                    break
    from IPython.display import display
    pd.options.display.max_columns = None
    for column in Scholar_df:
        flag =True
        for i in Scholar_df[column]:
            if i!=0:
                flag=False
        if flag:
            Scholar_df.drop(column,
      axis='columns', inplace=True)
    print('Csv File is ready')
    Scholar_df.to_csv(name+"_google_scholor_data.csv")
    return(Scholar_df)
df=scrap_goggle_scholor_page('rank4.csv')
df

Extracting Urls of Author's Google scholor page.....
['https://scholar.google.com/citations?hl=en&user=HBEjsgoAAAAJ', 'Not Found', 'Not Found', 'https://scholar.google.com/citations?hl=en&user=qxfa4OoAAAAJ', 'https://scholar.google.com/citations?hl=en&user=A8iq1jsAAAAJ', 'https://scholar.google.com/citations?hl=en&user=6nqXjKIAAAAJ', 'https://scholar.google.com/citations?hl=en&user=ucr5WoUAAAAJ', 'https://scholar.google.com/citations?hl=en&user=ayH_vVoAAAAJ', 'https://scholar.google.com/citations?hl=en&user=oMY7SmAAAAAJ', 'https://scholar.google.com/citations?hl=en&user=Vch_do8AAAAJ']
Extracting Data from Google Scholor Page......
10
10
10
10
Creating Csv File.......
Csv File is ready


,Author_URL,CITATIONS_ALL,HINDEX_ALL,I10_INDEX_ALL,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000,1999,1998,1997,1996,1995,1994,1993,1992,1991,1990,1989,1988
0,https://scholar.google.com/citations?hl=en&use...,186,7,5,15,38,36,31,13,21,12,14,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Not Found,None,None,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Not Found,None,None,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,https://scholar.google.com/citations?hl=en&use...,8516,41,88,93,291,337,390,427,493,487,607,601,605,593,594,527,361,431,356,285,285,193,142,99,67,32,22,0,0,0,0,0,0,0,0,0,0
4,https://scholar.google.com/citations?hl=en&use...,13566,62,114,754,1969,2149,1645,1328,1294,999,837,680,500,364,261,200,142,109,49,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,https://scholar.google.com/citations?hl=en&use...,62627,132,371,1488,4330,4435,4079,3938,3569,3493,3885,3364,2984,2956,2664,2621,2208,2182,1972,1660,1424,1022,1078,844,817,600,636,523,449,317,352,263,260,227,164,0,0
6,https://scholar.google.com/citations?hl=en&use...,5547,27,39,296,776,763,654,602,528,545,411,323,188,126,52,44,53,43,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,https://scholar.google.com/citations?hl=en&use...,266,7,7,30,70,56,37,22,22,15,8,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,https://scholar.google.com/citations?hl=en&use...,6857,33,48,197,493,540,435,469,478,463,407,490,466,473,430,302,308,202,96,105,62,62,70,67,73,68,51,0,0,0,0,0,0,0,0,0,0
9,https://scholar.google.com/citations?hl=en&use...,62631,118,222,1410,4039,3916,4085,3878,3713,4023,3894,3409,3045,2998,2612,2716,2413,2032,1827,1463,1631,1007,983,869,667,607,507,465,444,412,367,308,316,277,272,198,213


In [53]:
df

""


In [35]:
author_Citation=[]
author_HINDEX_ALL=[]
author_I10_INDEX_ALL=[]
author_year_list=[]
author_year_value_list=[]
for i in range(len(author_googleScholar_page_url)):
    if author_googleScholar_page_url[i]=='Not Found':
        author_Citation.append(None)
        author_HINDEX_ALL.append(None)
        author_I10_INDEX_ALL.append(None)
        author_year_list.append(None)
        author_year_value_list.append(None)
        continue
    scrap_author_googleScholar_data(author_googleScholar_page_url[i])

In [41]:
Scholar_df = pd.DataFrame({'Author_URL':author_googleScholar_page_url,'CITATIONS_ALL':author_Citation,'HINDEX_ALL':author_HINDEX_ALL,'I10_INDEX_ALL': author_I10_INDEX_ALL})
Starting_year=2021
for i in range(2021-1979):
    indx=str(Starting_year)
    Scholar_df[indx]=0
    Starting_year-=1
for column in Scholar_df:
    for i in range(len(author_year_list)):
        if author_year_list[i]==None:
            continue
        for j in range(len(author_year_list[i])):
            if column==author_year_list[i][j]:
                columnSeriesObj = Scholar_df[column]
                columnSeriesObj.values[i]=author_year_value_list[i][j]
                break
from IPython.display import display
pd.options.display.max_columns = None
for column in Scholar_df:
    flag =True
    for i in Scholar_df[column]:
        if i!=0:
            flag=False
    if flag:
        Scholar_df.drop(column,
  axis='columns', inplace=True)
Scholar_df

,Author_URL,CITATIONS_ALL,HINDEX_ALL,I10_INDEX_ALL,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000,1999,1998,1997,1996,1995,1994,1993,1992,1991,1990,1989,1988,1987,1986,1985,1984
0,https://scholar.google.com/citations?hl=en&use...,186,7,5,15,38,36,31,13,21,12,14,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Not Found,None,None,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Not Found,None,None,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,https://scholar.google.com/citations?hl=en&use...,8516,41,88,93,291,337,390,427,493,487,607,601,605,593,594,527,361,431,356,285,285,193,142,99,67,32,22,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,https://scholar.google.com/citations?hl=en&use...,13566,62,114,754,1969,2149,1645,1328,1294,999,837,680,500,364,261,200,142,109,49,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,https://scholar.google.com/citations?hl=en&use...,62627,132,371,1488,4330,4435,4079,3938,3569,3493,3885,3364,2984,2956,2664,2621,2208,2182,1972,1660,1424,1022,1078,844,817,600,636,523,449,317,352,263,260,227,164,0,0,0,0,0,0
6,https://scholar.google.com/citations?hl=en&use...,5547,27,39,296,776,763,654,602,528,545,411,323,188,126,52,44,53,43,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,https://scholar.google.com/citations?hl=en&use...,266,7,7,30,70,56,37,22,22,15,8,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,https://scholar.google.com/citations?hl=en&use...,6857,33,48,197,493,540,435,469,478,463,407,490,466,473,430,302,308,202,96,105,62,62,70,67,73,68,51,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,https://scholar.google.com/citations?hl=en&use...,62631,118,222,1410,4039,3916,4085,3878,3713,4023,3894,3409,3045,2998,2612,2716,2413,2032,1827,1463,1631,1007,983,869,667,607,507,465,444,412,367,308,316,277,272,198,213,0,0,0,0
